In [1]:
"""
Example script that scrapes data from the IEM ASOS download service
"""
from __future__ import print_function
import json
import time
import datetime
from urllib.request import urlopen

# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"


def download_data(uri):
    """Fetch the data from the IEM

    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.

    Args:
      uri (string): URL to fetch

    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read()
            if data is not None and not data.startswith(b'ERROR'):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""


def main():
    """Our main method"""
    # timestamps in UTC to request data for
    startts = datetime.datetime(2015, 7, 31)
    endts = datetime.datetime(2017, 7, 31)

    service = SERVICE + "data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&"

    service += startts.strftime('year1=%Y&month1=%m&day1=%d&')
    service += endts.strftime('year2=%Y&month2=%m&day2=%d&')

    #states = """AK AL AR AZ CA CO CT DE FL GA HI IA ID IL IN KS KY LA MA MD ME
    # MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT
    # WA WI WV WY"""
    
    states = """GA CO CA IL MN MI UT WA TX NV AZ NJ MD FL NC MA NY"""
    # IEM quirk to have Iowa AWOS sites in its own labeled network
    networks = ['AWOS']
    for state in states.split():
        networks.append("%s_ASOS" % (state,))

    for network in networks:
        # Get metadata
        uri = ("https://mesonet.agron.iastate.edu/"
               "geojson/network/%s.geojson") % (network,)
        print(uri)
        data = urlopen(uri)
        jdict = json.load(data)
        for site in jdict['features']:
            faaid = site['properties']['sid']
            sitename = site['properties']['sname']
            uri = '%s&station=%s' % (service, faaid)
            print(uri)
            print(('Network: %s Downloading: %s [%s]'
                   ) % (network, sitename, faaid))
            data = download_data(uri)
            outfn = '%s_%s_%s.txt' % (faaid, startts.strftime("%Y%m%d%H%M"),
                                      endts.strftime("%Y%m%d%H%M"))
            out = open(outfn, 'w')
            data = str(data,'utf-8')
            out.write(data)
            out.close()


if __name__ == '__main__':
    main()

https://mesonet.agron.iastate.edu/geojson/network/AWOS.geojson
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=AXA
Network: AWOS Downloading: ALGONA [AXA]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=IKV
Network: AWOS Downloading: ANKENY [IKV]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=AIO
Network: AWOS Downloading: ATLANTIC [AIO]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=ADU
Network: AWOS Downloading: AUDUBON [ADU]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=E

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=RDK
Network: AWOS Downloading: RED OAK [RDK]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=SHL
Network: AWOS Downloading: SHELDON [SHL]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=SDA
Network: AWOS Downloading: SHENANDOAH MUNI [SDA]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=SLB
Network: AWOS Downloading: Storm Lake [SLB]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=DNN
Network: GA_ASOS Downloading: DALTON MUNI AIRPORT [DNN]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=MGE
Network: GA_ASOS Downloading: DOBBINS AFB/MARIETT [MGE]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=17J
Network: GA_ASOS Downloading: Donalsonville [17J]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=DQH
Network: GA_ASOS Downloading: DOUGLAS_MUNI_ARPT [DQH]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&for

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=PIM
Network: GA_ASOS Downloading: Pine Mountain [PIM]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=RMG
Network: GA_ASOS Downloading: ROME/RUSSELL(RAMOS) [RMG]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=OKZ
Network: GA_ASOS Downloading: Sandersville  [OKZ]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=SAV
Network: GA_ASOS Downloading: SAVANNAH MUNICIPAL [SAV]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=o

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=CAG
Network: CO_ASOS Downloading: CRAIG-MOFFAT(AMOS) [CAG]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=RCV
Network: CO_ASOS Downloading: Del Norte [RCV]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=AJZ
Network: CO_ASOS Downloading: Delta [AJZ]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=APA
Network: CO_ASOS Downloading: DENVER/CENTENNIAL [APA]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlo

...

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=TDR
Network: FL_ASOS Downloading: Panama City Beach - Drone Runway [TDR]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=COF
Network: FL_ASOS Downloading: PATRICK AFB/COCOA B [COF]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=NPA
Network: FL_ASOS Downloading: PENSACOLA NAS [NPA]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=PNS
Network: FL_ASOS Downloading: PENSACOLA REGIONAL [PNS]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=GEV
Network: NC_ASOS Downloading: ASHE_COUNTY_ARPT [GEV]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=AVL
Network: NC_ASOS Downloading: ASHEVILLE_MUNICIPAL_(ASOS) [AVL]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=NJM
Network: NC_ASOS Downloading: Bogue Field [NJM]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=BUY
Network: NC_ASOS Downloading: BURLINGTON-ALAMANCE_REG_(ASOS) [BUY]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=MQI
Network: NC_ASOS Downloading: MANTEO/DARE_CO_RGNL [MQI]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=EQY
Network: NC_ASOS Downloading: MONROE_AIRPORT_(ASOS) [EQY]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=MRN
Network: NC_ASOS Downloading: MORGANTON-LENOIR_AIRPORT [MRN]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=MWK
Network: NC_ASOS Downloading: MOUNT_AIRY/SURRY_COUNTY_APRT [MWK]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=INT
Network: NC_ASOS Downloading: WINSTON-SALEM/SMITH_(ASOS) [INT]
https://mesonet.agron.iastate.edu/geojson/network/MA_ASOS.geojson
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=BVY
Network: MA_ASOS Downloading: BEVERLY MUNICIPAL [BVY]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=BOS
Network: MA_ASOS Downloading: BOSTON/LOGAN INTL [BOS]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=CQX
Network: MA_ASOS Downloading: CHATHAM MUNI ARPT [CQX]
http:

http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=GFL
Network: NY_ASOS Downloading: GLEN FALLS/WARREN [GFL]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=RME
Network: NY_ASOS Downloading: GRIFFISS AFB/ROME [RME]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=ISP
Network: NY_ASOS Downloading: ISLIP/MACARTHUR [ISP]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma&latlon=yes&year1=2015&month1=07&day1=31&year2=2017&month2=07&day2=31&&station=ITH
Network: NY_ASOS Downloading: Ithaca [ITH]
http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=onlycomma